# Voice-to-Text Endpoint Testing (Mic + File)
**Author:** Venkatesh Manikantan, Senior Associate, PwC India  
**Client:** Nokia  

Tests all three voice endpoints:
1. **`POST /transcribe`** — single-shot (file upload)
2. **`POST /transcribe/stream`** — SSE streaming (file upload)
3. **`WS /ws/transcribe`** — WebSocket real-time (live mic, ffmpeg decoded)

### Prerequisites
```bash
pip install sounddevice scipy websockets requests
```
Also make sure **ffmpeg** is on your PATH (the WS endpoint uses it to decode browser audio).

In [ ]:
import requests
import json
import time
import wave
import io
import struct
import asyncio
from pathlib import Path

import sounddevice as sd
import numpy as np

BASE_URL = "http://localhost:8000/api/v1/voice"
WS_URL   = "ws://localhost:8000/api/v1/voice/ws/transcribe"
LANGUAGE = "en-US"

# Audio recording settings
SAMPLE_RATE = 16000   # 16kHz — optimal for Azure Speech
CHANNELS    = 1       # Mono
DTYPE       = "int16" # 16-bit PCM

print(f"API Base : {BASE_URL}")
print(f"WS  URL  : {WS_URL}")
print(f"Language : {LANGUAGE}")
print(f"Devices  : {sd.query_devices()}")

ModuleNotFoundError: No module named 'sounddevice'

---
## Helper: Record from Microphone
Records audio from your default mic and returns a WAV file as bytes.

In [ ]:
def record_audio(duration_seconds: int = 5) -> bytes:
    """
    Record from the default microphone and return WAV bytes.
    
    Args:
        duration_seconds: How long to record (default 5s)
    Returns:
        WAV file as bytes (16kHz, 16-bit, mono)
    """
    print(f"Recording for {duration_seconds} seconds... Speak now!")
    audio_data = sd.rec(
        int(duration_seconds * SAMPLE_RATE),
        samplerate=SAMPLE_RATE,
        channels=CHANNELS,
        dtype=DTYPE,
    )
    sd.wait()  # block until recording is done
    print("Recording complete.")

    # Convert numpy array to WAV bytes
    buf = io.BytesIO()
    with wave.open(buf, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(2)  # 16-bit = 2 bytes
        wf.setframerate(SAMPLE_RATE)
        wf.writeframes(audio_data.tobytes())
    
    wav_bytes = buf.getvalue()
    print(f"WAV size: {len(wav_bytes):,} bytes ({duration_seconds}s @ {SAMPLE_RATE}Hz)")
    return wav_bytes

---
## 1. Test Regular Transcription (`POST /transcribe`)
Record from mic, upload as WAV, get single-shot result.

In [ ]:
# Record 5 seconds from your mic
wav_bytes = record_audio(duration_seconds=5)

In [ ]:
# Send to regular /transcribe endpoint
start = time.time()
resp = requests.post(
    f"{BASE_URL}/transcribe",
    files={"audio": ("recording.wav", wav_bytes, "audio/wav")},
    params={"language": LANGUAGE},
)
elapsed = time.time() - start

print(f"Status  : {resp.status_code}")
print(f"Latency : {elapsed:.2f}s")

if resp.status_code == 200:
    data = resp.json()
    print(f"Result  : {data['status']}")
    print(f"Duration: {data.get('duration_seconds')}s")
    print(f"Text    : {data['text']}")
else:
    print(f"Error   : {resp.text}")

---
## 2. Test Streaming Transcription (`POST /transcribe/stream`)
Same recording, but get partial results streamed back via SSE.

In [ ]:
# Record fresh audio (or reuse wav_bytes from above)
wav_bytes_stream = record_audio(duration_seconds=8)

In [ ]:
# Send to SSE /transcribe/stream endpoint
start = time.time()
resp = requests.post(
    f"{BASE_URL}/transcribe/stream",
    files={"audio": ("recording.wav", wav_bytes_stream, "audio/wav")},
    params={"language": LANGUAGE},
    stream=True,
)

if resp.status_code != 200:
    print(f"Error {resp.status_code}: {resp.text}")
else:
    print("SSE Events:\n")
    current_event = None
    current_data = None
    all_recognized = []

    for line in resp.iter_lines(decode_unicode=True):
        if line is None:
            continue
        if line.startswith("event:"):
            current_event = line[len("event:"):].strip()
        elif line.startswith("data:"):
            current_data = line[len("data:"):].strip()
        elif line == "":
            if current_event and current_data:
                payload = json.loads(current_data)
                elapsed = time.time() - start
                if current_event == "recognizing":
                    print(f"  [{elapsed:6.2f}s] RECOGNIZING : {payload.get('text', '')}")
                elif current_event == "recognized":
                    text = payload.get('text', '')
                    all_recognized.append(text)
                    print(f"  [{elapsed:6.2f}s] RECOGNIZED  : {text}")
                elif current_event == "done":
                    print(f"  [{elapsed:6.2f}s] DONE")
                elif current_event == "error":
                    print(f"  [{elapsed:6.2f}s] ERROR: {payload}")
            current_event = None
            current_data = None

    print(f"\nFull transcription: {' '.join(all_recognized)}")

---
## 3. Test Real-Time WebSocket (`WS /ws/transcribe`)
Live mic streaming — sends raw PCM audio directly over WebSocket using `audio_format=pcm`.  
No ffmpeg encoding needed on the client side (the server accepts raw 16kHz/16-bit/mono PCM).

> **Note:** The browser frontend uses `audio_format=webm` (default) which goes through ffmpeg on the server.  
> For notebook testing we use `audio_format=pcm` to skip that step entirely.

In [3]:
import websockets

async def test_ws_realtime(duration_seconds: int = 8, language: str = "en-US"):
    """
    Stream live mic audio to the WebSocket endpoint using raw PCM mode.
    
    Uses audio_format=pcm so the server pushes bytes directly to Azure SDK
    without needing ffmpeg decoding. Perfect for notebook testing.
    """
    # Connect with audio_format=pcm — server skips ffmpeg, expects raw 16kHz/16-bit/mono
    url = f"{WS_URL}?language={language}&audio_format=pcm"
    print(f"Connecting to {url} ...")

    async with websockets.connect(url) as ws_conn:
        print(f"Connected! Recording for {duration_seconds}s — speak now!\n")

        async def send_audio():
            """Record from mic and send raw PCM chunks over WebSocket."""
            # Record full audio
            audio_data = sd.rec(
                int(duration_seconds * SAMPLE_RATE),
                samplerate=SAMPLE_RATE,
                channels=CHANNELS,
                dtype=DTYPE,
            )
            sd.wait()
            print("  Recording complete, sending audio...")

            # Send raw PCM in 3200-byte chunks (100ms of 16kHz 16-bit mono)
            pcm_bytes = audio_data.tobytes()
            CHUNK_SIZE = 3200
            offset = 0
            while offset < len(pcm_bytes):
                chunk = pcm_bytes[offset : offset + CHUNK_SIZE]
                await ws_conn.send(chunk)
                offset += CHUNK_SIZE
                await asyncio.sleep(0.02)  # small delay to avoid flooding

            await asyncio.sleep(0.5)  # let server process remaining audio
            await ws_conn.send("STOP")
            print("  Sent all audio + STOP")

        async def receive_results():
            """Read transcription results from the WebSocket."""
            start = time.time()
            all_recognized = []
            try:
                async for message in ws_conn:
                    data = json.loads(message)
                    elapsed = time.time() - start
                    event = data.get("event", "unknown")

                    if event == "recognizing":
                        print(f"  [{elapsed:6.2f}s] RECOGNIZING : {data.get('text', '')}")
                    elif event == "recognized":
                        text = data.get("text", "")
                        all_recognized.append(text)
                        print(f"  [{elapsed:6.2f}s] RECOGNIZED  : {text}")
                    elif event == "done":
                        print(f"  [{elapsed:6.2f}s] DONE")
                        break
                    elif event == "error":
                        print(f"  [{elapsed:6.2f}s] ERROR: {data}")
                        break
            except websockets.ConnectionClosed:
                print("  Connection closed.")

            print(f"\n  Full transcription: {' '.join(all_recognized)}")

        await asyncio.gather(send_audio(), receive_results())

    print("\nWebSocket session complete.")

ModuleNotFoundError: No module named 'websockets'

In [ ]:
# Run the real-time WebSocket test — speak for 8 seconds
await test_ws_realtime(duration_seconds=8, language="en-US")

---
## 4. Live Mic Streaming over WebSocket (Real-Time)
Streams audio from your microphone **as you speak** — no pre-recording.  
You'll see partial ("recognizing") results updating in real time, just like the browser experience.

Uses `sounddevice.InputStream` with a callback that feeds chunks directly into the WebSocket.  
Send **Ctrl+C** or let the timeout expire to stop.

In [ ]:
import queue as thread_queue

async def test_ws_live_mic(max_seconds: int = 30, language: str = "en-US"):
    """
    TRUE live mic streaming — audio is captured and sent in real-time.
    
    Unlike test_ws_realtime() which records first then sends, this function
    opens a sounddevice.InputStream that continuously feeds PCM chunks into
    the WebSocket as you speak. You see transcription results immediately.
    
    Args:
        max_seconds: Maximum duration before auto-stopping (default 30s).
        language: BCP-47 language code.
    """
    url = f"{WS_URL}?language={language}&audio_format=pcm"
    print(f"Connecting to {url} ...")

    # Thread-safe queue: sounddevice callback (separate thread) → async send task
    audio_queue = thread_queue.Queue()

    def audio_callback(indata, frames, time_info, status):
        """Called by sounddevice on its own thread for every audio chunk."""
        if status:
            print(f"  [audio] {status}")
        # indata is a numpy array — copy bytes into the queue
        audio_queue.put(bytes(indata))

    async with websockets.connect(url) as ws_conn:
        print(f"Connected! Speak now (up to {max_seconds}s). Results appear in real time.\n")
        print("=" * 60)

        stop_event = asyncio.Event()

        async def send_audio():
            """Pull PCM chunks from the thread-safe queue and send over WebSocket."""
            loop = asyncio.get_event_loop()
            start = time.time()

            # Open mic stream — 100ms chunks (1600 frames @ 16kHz = 3200 bytes)
            stream = sd.InputStream(
                samplerate=SAMPLE_RATE,
                channels=CHANNELS,
                dtype=DTYPE,
                blocksize=1600,       # 100ms of audio per callback
                callback=audio_callback,
            )
            stream.start()
            print("  [mic] Recording started...")

            try:
                while not stop_event.is_set():
                    # Check timeout
                    if time.time() - start >= max_seconds:
                        print(f"\n  [mic] Timeout ({max_seconds}s) reached.")
                        break

                    # Drain audio queue → WebSocket
                    try:
                        chunk = await loop.run_in_executor(
                            None, lambda: audio_queue.get(timeout=0.1)
                        )
                        await ws_conn.send(chunk)
                    except thread_queue.Empty:
                        continue
            finally:
                stream.stop()
                stream.close()
                # Drain any remaining audio
                while not audio_queue.empty():
                    try:
                        chunk = audio_queue.get_nowait()
                        await ws_conn.send(chunk)
                    except thread_queue.Empty:
                        break

                await asyncio.sleep(0.3)  # let server process last chunks
                await ws_conn.send("STOP")
                print("  [mic] Sent STOP")

        async def receive_results():
            """Read transcription results and display in real time."""
            start = time.time()
            all_recognized = []
            try:
                async for message in ws_conn:
                    data = json.loads(message)
                    elapsed = time.time() - start
                    event = data.get("event", "unknown")

                    if event == "recognizing":
                        # Overwrite current line for live partial updates
                        text = data.get("text", "")
                        print(f"\r  [{elapsed:6.2f}s]  ... {text}", end="", flush=True)
                    elif event == "recognized":
                        text = data.get("text", "")
                        all_recognized.append(text)
                        print(f"\r  [{elapsed:6.2f}s]  >>> {text}")
                    elif event == "done":
                        print(f"\n  [{elapsed:6.2f}s] DONE")
                        stop_event.set()
                        break
                    elif event == "error":
                        print(f"\n  [{elapsed:6.2f}s] ERROR: {data}")
                        stop_event.set()
                        break
            except websockets.ConnectionClosed:
                print("\n  Connection closed.")
                stop_event.set()

            print("=" * 60)
            print(f"\n  Full transcription:\n  {' '.join(all_recognized)}")

        await asyncio.gather(send_audio(), receive_results())

    print("\nLive session complete.")

Calculate the average time (in days) from 'GC Selected' to 'Construction NTP Accepted by GC' for projects in each region

In [5]:
# Run the real-time WebSocket test — speak for 8 seconds
await test_ws_live_mic(max_seconds=30, language="en-US")

NameError: name 'WS_URL' is not defined

---
## 5. Quick Health Check

---
## 4. Quick Health Check

In [ ]:
resp = requests.get("http://localhost:8000/health")
print(f"Health: {resp.json()}")